# Создание первого чатбота

Чат-бот— это программа, которая имитирует реальный разговор с пользователем. Чат-боты позволяют общаться с помощью текстовых или аудио сообщений на сайтах, в мессенджерах, мобильных приложениях 

Устанавливаем библиотеку python-telegram-bot. Ссылка на документацию https://github.com/python-telegram-bot/python-telegram-bot

In [1]:
!pip install python-telegram-bot --upgrade

In [1]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import dialogflow

Done! Congratulations on your new bot. You will find it at t.me/Evm_gb_bot. You can now add a description, about section and profile picture for your bot, see /help for a list of commands. By the way, when you've finished creating your cool bot, ping our Bot Support if you want a better username for it. Just make sure the bot is fully operational before you do this.

Use this token to access the HTTP API:
1486721342:AAFR7RPnqZHHZSjmRsBmQqaM2Kb5t8bJaes
Keep your token secure and store it safely, it can be used by anyone to control your bot.

For a description of the Bot API, see this page: https://core.telegram.org/bots/api

In [2]:
updater = Updater("1486721342:AAFR7RPnqZHHZSjmRsBmQqaM2Kb5t8bJaes", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'newbot-cfin-91c1d9900dfd.json'# скачнный JSON


DIALOGFLOW_PROJECT_ID = 'newbot-cfin' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'Evm_gb_bot'  # ID бота из телеграма

Переписываем функцию textMessage

In [3]:
def startCommand1(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

    
def textMessage1(update: Update, context: CallbackContext):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=update.message.text , language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text:
        update.message.reply_text(response.query_result.fulfillment_text)
    else:
        update.message.reply_text('Что?')

In [4]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand1))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage1))

# Start the Bot
updater.start_polling()
updater.idle()